In [23]:
import os
import gc
import re
import pickle
import datetime
from tqdm import tqdm
pd.set_option('display.max_columns', 100)

import numpy as np
import pandas as pd

from typing import Union

import seaborn
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

from scipy.stats import linregress

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

In [8]:
df = pd.read_pickle('/Users/rui/Documents/repositories/m5_forecasting_accuracy/v01000/features/add_weight.pkl')

In [25]:
print(df.shape)
df.tail()

(45204727, 76)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,quarter,week,day,dayofweek,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start,is_weekend,event_name_1_lag_t1,event_name_1_lag_t2,event_type_1_lag_t1,event_type_1_lag_t2,event_name_2_lag_t1,event_name_2_lag_t2,event_type_2_lag_t1,event_type_2_lag_t2,sell_price,release,price_MAX,price_MIN,price_STD,price_MEAN,price_NORM,price_NUNIQUE,item_NUNIQUE,price_momentum,sales_lag_t28,sales_lag_t29,sales_lag_t30,sales_lag_t32,sales_lag_t33,sales_lag_t34,sales_rolling_STD_t7,sales_rolling_STD_t30,sales_rolling_STD_t60,sales_rolling_STD_t90,sales_rolling_STD_t180,sales_rolling_MEAN_t7,sales_rolling_MEAN_t30,sales_rolling_MEAN_t60,sales_rolling_MEAN_t90,sales_rolling_MEAN_t180,sales_rolling_MIN_t7,sales_rolling_MIN_t30,sales_rolling_MIN_t60,sales_rolling_MAX_t7,sales_rolling_MAX_t30,sales_rolling_MAX_t60,sales_rolling_SKEW_t30,sales_rolling_KURT_t30,sell_price_lag_t28,sell_price_price_change_t365,sell_price_rolling_price_std_t7,sell_price_rolling_price_std_t30,weight
46881672,FOODS_3_823_WI_3_validation,1432,2,0,9,2,d_1941,0,2016-05-22,11617,5,19,2,3,1,0,0,0,2,20,22,6,0,0,0,0,0,0,1,19.0,19.0,2.0,2.0,3.0,3.0,1.0,1.0,2.980469,11104.0,2.980469,2.480469,0.171875,2.802734,1.000000,5.0,206.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.488037,0.550781,0.571289,0.848633,0.987305,0.285645,0.199951,0.250000,0.433350,0.616699,0.0,0.0,0.0,1.0,2.0,2.0,2.757812,6.730469,2.980469,0.0,0.000000e+00,0.000000,0.142822
46881673,FOODS_3_824_WI_3_validation,1433,2,0,9,2,d_1941,0,2016-05-22,11617,5,19,2,3,1,0,0,0,2,20,22,6,0,0,0,0,0,0,1,19.0,19.0,2.0,2.0,3.0,3.0,1.0,1.0,2.480469,11104.0,2.679688,2.000000,0.252930,2.507812,0.925781,4.0,135.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.377930,0.535156,0.404541,0.336914,0.242798,0.142822,0.300049,0.150024,0.099976,0.049988,0.0,0.0,0.0,1.0,2.0,2.0,1.621094,1.950195,2.480469,0.0,5.960464e-08,0.206665,0.285645
46881674,FOODS_3_825_WI_3_validation,1434,2,0,9,2,d_1941,0,2016-05-22,11617,5,19,2,3,1,0,0,0,2,20,22,6,0,0,0,0,0,0,1,19.0,19.0,2.0,2.0,3.0,3.0,1.0,1.0,3.980469,11104.0,4.378906,3.980469,0.187866,4.117188,0.909180,3.0,150.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.786621,1.136719,1.056641,0.972168,0.941895,0.571289,0.866699,1.033203,0.899902,0.761230,0.0,0.0,0.0,2.0,4.0,4.0,1.485352,1.986328,3.980469,0.0,0.000000e+00,0.000000,0.500000
46881675,FOODS_3_826_WI_3_validation,1435,2,0,9,2,d_1941,0,2016-05-22,11617,5,19,2,3,1,0,0,0,2,20,22,6,0,0,0,0,0,0,1,19.0,19.0,2.0,2.0,3.0,3.0,1.0,1.0,1.280273,11328.0,1.280273,1.280273,0.000000,1.280273,1.000000,1.0,44.0,1.0,3.0,1.0,3.0,1.0,0.0,0.0,1.344727,1.172852,1.065430,1.101562,1.550781,1.142578,1.066406,1.016602,1.022461,1.338867,0.0,0.0,0.0,3.0,4.0,4.0,0.962891,-0.021317,1.280273,0.0,0.000000e+00,0.000000,0.571289
46881676,FOODS_3_827_WI_3_validation,1436,2,0,9,2,d_1941,0,2016-05-22,11617,5,19,2,3,1,0,0,0,2,20,22,6,0,0,0,0,0,0,1,19.0,19.0,2.0,2.0,3.0,3.0,1.0,1.0,1.000000,11408.0,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,142.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.821289,1.770508,1.860352,1.741211,0.000000,1.166992,1.500000,1.566406,1.510742,0.0,0.0,0.0,0.0,7.0,7.0,1.680664,2.517578,1.000000,0.0,0.000000e+00,0.000000,0.392822
